In [1]:
import torch
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import numpy as np
import torch.nn as nn
from math import *

In [2]:
torch.cuda.set_device(1)

In [3]:
torch.set_default_tensor_type('torch.DoubleTensor') # 设置浮点类型为 torch.float64

In [4]:
# 定义激活函数: swish(x)
def acti(x):
    return x*torch.sigmoid(x) 

In [5]:
# 定义网络结构
class DeepRitzNet(torch.nn.Module):
    def __init__(self, input_width, layer_width):
        super(DeepRitzNet, self).__init__()
        self.linear_in = torch.nn.Linear(input_width, layer_width)
        self.linear1 = torch.nn.Linear(layer_width, layer_width)
        self.linear2 = torch.nn.Linear(layer_width, layer_width)
        self.linear3 = torch.nn.Linear(layer_width, layer_width)
        self.linear4 = torch.nn.Linear(layer_width, layer_width)
        self.linear5 = torch.nn.Linear(layer_width, layer_width)
        self.linear6 = torch.nn.Linear(layer_width, layer_width)
        self.linear_out = torch.nn.Linear(layer_width, 1)

    def forward(self, x):
        y = self.linear_in(x) # fully connect layer
        y = y + acti(self.linear2(acti(self.linear1(y)))) # residual block 1
        y = y + acti(self.linear4(acti(self.linear3(y)))) # residual block 2
        y = y + acti(self.linear6(acti(self.linear5(y)))) # residual block 3
        output = self.linear_out(y) # fully connect layer
        return output

In [6]:
dimension = 4

In [7]:
# exact solution
def u_ex(x):  
    x_temp = torch.cos(pi*x)
    u_x = (x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return u_x

In [8]:
def f(x):
    x_temp = torch.cos(pi*x)
    f_x = 2*pi**2*(x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return f_x

In [9]:
def g(x):  
    x_temp = torch.cos(pi*x)
    g_x = (x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return g_x

In [10]:
# Deep Ritz Method
def DRM(x):
    u_hat = model(x)
    ux = torch.zeros(x.size()[0], dimension).cuda()
    step_size = 0.0001
    for i in range(dimension):
        dx = torch.zeros(x.size()[0], dimension).cuda()
        dx[:, i] = torch.ones(x.size()[0])
        ux[:, i] = (model(x+step_size*dx) - model(x-step_size*dx))[:,0]/step_size/2
    uxsq = (torch.sum(ux**2, dim = 1)).reshape([x.size()[0], 1]) # dim = 1 按行求和
    f_temp = f(x)
    part_1 = torch.sum(0.5 * uxsq + 0.5*pi**2*u_hat**2 - f_temp*u_hat)/x.size()[0]
   
    Nb = 100
    xa1 = torch.rand(Nb, dimension).cuda() 
    xa1[:, 0] = torch.zeros(Nb)
    xa2 = torch.rand(Nb, dimension).cuda() 
    xa2[:, 0] = torch.ones(Nb)
    xb1 = torch.rand(Nb, dimension).cuda()
    xb1[:, 1] = torch.zeros(Nb)
    xb2 = torch.rand(Nb, dimension).cuda()
    xb2[:, 1] = torch.ones(Nb)
    xc1 = torch.rand(Nb, dimension).cuda()
    xc1[:, 2] = torch.zeros(Nb)
    xc2 = torch.rand(Nb, dimension).cuda()
    xc2[:, 2] = torch.ones(Nb)
    xd1 = torch.rand(Nb, dimension).cuda()
    xd1[:, 3] = torch.zeros(Nb)
    xd2 = torch.rand(Nb, dimension).cuda()
    xd2[:, 3] = torch.ones(Nb)
    dx1 = torch.zeros(xb1.size()[0], dimension).cuda()
    dx1[:, 0] = torch.ones(xb1.size()[0])
    dx2 = torch.zeros(xb1.size()[0], dimension).cuda()
    dx2[:, 1] = torch.ones(xb1.size()[0])
    dx3 = torch.zeros(xb1.size()[0], dimension).cuda()
    dx3[:, 2] = torch.ones(xb1.size()[0])
    dx4 = torch.zeros(xb1.size()[0], dimension).cuda()
    dx4[:, 3] = torch.ones(xb1.size()[0])
    uxa1 = (((model(xa1+step_size*dx1) - model(xa1-step_size*dx1))/step_size/2 * (1.0) + model(xa1) - g(xa1))**2).cuda()
    uxa2 = (((model(xa2+step_size*dx1) - model(xa2-step_size*dx1))/step_size/2 * (-1.0) + model(xa2) - g(xa2))**2).cuda()
    uxb1 = (((model(xb1+step_size*dx2) - model(xb1-step_size*dx2))/step_size/2 * (1.0) + model(xb1) - g(xb1))**2).cuda()
    uxb2 = (((model(xb2+step_size*dx2) - model(xb2-step_size*dx2))/step_size/2 * (-1.0) + model(xb2) - g(xb2))**2).cuda()
    uxc1 = (((model(xc1+step_size*dx3) - model(xc1-step_size*dx3))/step_size/2 * (1.0) + model(xc1) - g(xc1))**2).cuda()
    uxc2 = (((model(xc2+step_size*dx3) - model(xc2-step_size*dx3))/step_size/2 * (-1.0) + model(xc2) - g(xc2))**2).cuda()
    uxd1 = (((model(xd1+step_size*dx4) - model(xd1-step_size*dx4))/step_size/2 * (1.0) + model(xd1) - g(xd1))**2).cuda()
    uxd2 = (((model(xd2+step_size*dx4) - model(xd2-step_size*dx4))/step_size/2 * (-1.0) + model(xd2) - g(xd2))**2).cuda()
    
    part_2 = torch.sum((uxa1 + uxa2 + uxb1 + uxb2 + uxc1 + uxc2 + uxd1 + uxd2)[:,0])/xa1.size()[0]
    lambda1 = 500
    return part_1 + lambda1 * part_2 / 8

In [11]:
Data_size = 4000
def Gendata():
    x = torch.rand(Data_size, dimension)
    return x.cuda()

In [12]:
# 正态分布初始化参数
def initparam(model, sigma):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            m.weight.data.normal_(0, sigma)
    return model

In [13]:
model = DeepRitzNet(dimension, 8)
model = initparam(model, 0.5)

In [14]:
device = torch.device("cuda:1" )
model.to(device)

DeepRitzNet(
  (linear_in): Linear(in_features=4, out_features=8, bias=True)
  (linear1): Linear(in_features=8, out_features=8, bias=True)
  (linear2): Linear(in_features=8, out_features=8, bias=True)
  (linear3): Linear(in_features=8, out_features=8, bias=True)
  (linear4): Linear(in_features=8, out_features=8, bias=True)
  (linear5): Linear(in_features=8, out_features=8, bias=True)
  (linear6): Linear(in_features=8, out_features=8, bias=True)
  (linear_out): Linear(in_features=8, out_features=1, bias=True)
)

In [15]:
import torch.optim as optim
import torch.nn as nn
import time

In [16]:
def relative_error():
    x = Gendata()
    predict = model(x)
    exact = u_ex(x)
    value = torch.sqrt(torch.sum((predict - exact )**2))/torch.sqrt(torch.sum((exact )**2))
    return value

In [17]:
traintime = 20000
error_save = np.zeros(traintime)
optimizer = optim.Adam(model.parameters())

In [18]:
time_start = time.time()
for i in range(traintime):
    optimizer.zero_grad()
    x = Gendata()
    x.requires_grad = True
    losses = DRM(x)
    losses.backward()
    optimizer.step()
    error = relative_error()
    error_save[i] = float(error)
    
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", losses.detach())
        print("current relative error is: ", error.detach())
        np.save("DRM_relative_error_4D_Robin_batch_size_4000.npy", error_save)
np.save("DRM_relative_error_4D_Robin_batch_size_4000.npy", error_save)
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(6469.6156, device='cuda:1')
current relative error is:  tensor(2.3230, device='cuda:1')
current epoch is:  50
current loss is:  tensor(1519.9935, device='cuda:1')
current relative error is:  tensor(0.7223, device='cuda:1')
current epoch is:  100
current loss is:  tensor(1127.3795, device='cuda:1')
current relative error is:  tensor(0.7364, device='cuda:1')
current epoch is:  150
current loss is:  tensor(903.7141, device='cuda:1')
current relative error is:  tensor(0.7179, device='cuda:1')
current epoch is:  200
current loss is:  tensor(631.9377, device='cuda:1')
current relative error is:  tensor(0.7377, device='cuda:1')
current epoch is:  250
current loss is:  tensor(565.1786, device='cuda:1')
current relative error is:  tensor(0.7842, device='cuda:1')
current epoch is:  300
current loss is:  tensor(512.8391, device='cuda:1')
current relative error is:  tensor(0.8402, device='cuda:1')
current epoch is:  350
current loss is:  tensor(439.293

current epoch is:  3050
current loss is:  tensor(-12.7716, device='cuda:1')
current relative error is:  tensor(0.3794, device='cuda:1')
current epoch is:  3100
current loss is:  tensor(-11.7036, device='cuda:1')
current relative error is:  tensor(0.3681, device='cuda:1')
current epoch is:  3150
current loss is:  tensor(-12.6622, device='cuda:1')
current relative error is:  tensor(0.3674, device='cuda:1')
current epoch is:  3200
current loss is:  tensor(-12.4438, device='cuda:1')
current relative error is:  tensor(0.3551, device='cuda:1')
current epoch is:  3250
current loss is:  tensor(-12.3925, device='cuda:1')
current relative error is:  tensor(0.3515, device='cuda:1')
current epoch is:  3300
current loss is:  tensor(-12.2663, device='cuda:1')
current relative error is:  tensor(0.3483, device='cuda:1')
current epoch is:  3350
current loss is:  tensor(-13.3695, device='cuda:1')
current relative error is:  tensor(0.3459, device='cuda:1')
current epoch is:  3400
current loss is:  tensor

current epoch is:  6100
current loss is:  tensor(-17.3065, device='cuda:1')
current relative error is:  tensor(0.1682, device='cuda:1')
current epoch is:  6150
current loss is:  tensor(-16.5072, device='cuda:1')
current relative error is:  tensor(0.1679, device='cuda:1')
current epoch is:  6200
current loss is:  tensor(-18.4075, device='cuda:1')
current relative error is:  tensor(0.1664, device='cuda:1')
current epoch is:  6250
current loss is:  tensor(-17.2900, device='cuda:1')
current relative error is:  tensor(0.1646, device='cuda:1')
current epoch is:  6300
current loss is:  tensor(-17.8796, device='cuda:1')
current relative error is:  tensor(0.1653, device='cuda:1')
current epoch is:  6350
current loss is:  tensor(-18.7811, device='cuda:1')
current relative error is:  tensor(0.1623, device='cuda:1')
current epoch is:  6400
current loss is:  tensor(-18.8694, device='cuda:1')
current relative error is:  tensor(0.1650, device='cuda:1')
current epoch is:  6450
current loss is:  tensor

current epoch is:  9150
current loss is:  tensor(-19.8841, device='cuda:1')
current relative error is:  tensor(0.1118, device='cuda:1')
current epoch is:  9200
current loss is:  tensor(-19.1854, device='cuda:1')
current relative error is:  tensor(0.1094, device='cuda:1')
current epoch is:  9250
current loss is:  tensor(-18.3392, device='cuda:1')
current relative error is:  tensor(0.1109, device='cuda:1')
current epoch is:  9300
current loss is:  tensor(-19.3507, device='cuda:1')
current relative error is:  tensor(0.1110, device='cuda:1')
current epoch is:  9350
current loss is:  tensor(-19.0362, device='cuda:1')
current relative error is:  tensor(0.1108, device='cuda:1')
current epoch is:  9400
current loss is:  tensor(-19.0610, device='cuda:1')
current relative error is:  tensor(0.1088, device='cuda:1')
current epoch is:  9450
current loss is:  tensor(-18.0569, device='cuda:1')
current relative error is:  tensor(0.1084, device='cuda:1')
current epoch is:  9500
current loss is:  tensor

current epoch is:  12150
current loss is:  tensor(-19.5901, device='cuda:1')
current relative error is:  tensor(0.0862, device='cuda:1')
current epoch is:  12200
current loss is:  tensor(-19.0419, device='cuda:1')
current relative error is:  tensor(0.0859, device='cuda:1')
current epoch is:  12250
current loss is:  tensor(-20.9028, device='cuda:1')
current relative error is:  tensor(0.0861, device='cuda:1')
current epoch is:  12300
current loss is:  tensor(-19.2489, device='cuda:1')
current relative error is:  tensor(0.0831, device='cuda:1')
current epoch is:  12350
current loss is:  tensor(-19.1386, device='cuda:1')
current relative error is:  tensor(0.0820, device='cuda:1')
current epoch is:  12400
current loss is:  tensor(-19.2089, device='cuda:1')
current relative error is:  tensor(0.0810, device='cuda:1')
current epoch is:  12450
current loss is:  tensor(-19.3886, device='cuda:1')
current relative error is:  tensor(0.0826, device='cuda:1')
current epoch is:  12500
current loss is:

current epoch is:  15150
current loss is:  tensor(-18.9385, device='cuda:1')
current relative error is:  tensor(0.0679, device='cuda:1')
current epoch is:  15200
current loss is:  tensor(-19.6278, device='cuda:1')
current relative error is:  tensor(0.0709, device='cuda:1')
current epoch is:  15250
current loss is:  tensor(-19.5722, device='cuda:1')
current relative error is:  tensor(0.0681, device='cuda:1')
current epoch is:  15300
current loss is:  tensor(-19.2886, device='cuda:1')
current relative error is:  tensor(0.0706, device='cuda:1')
current epoch is:  15350
current loss is:  tensor(-19.0294, device='cuda:1')
current relative error is:  tensor(0.0688, device='cuda:1')
current epoch is:  15400
current loss is:  tensor(-18.9304, device='cuda:1')
current relative error is:  tensor(0.0705, device='cuda:1')
current epoch is:  15450
current loss is:  tensor(-20.0860, device='cuda:1')
current relative error is:  tensor(0.0684, device='cuda:1')
current epoch is:  15500
current loss is:

current epoch is:  18150
current loss is:  tensor(-19.5953, device='cuda:1')
current relative error is:  tensor(0.0607, device='cuda:1')
current epoch is:  18200
current loss is:  tensor(-19.5436, device='cuda:1')
current relative error is:  tensor(0.0606, device='cuda:1')
current epoch is:  18250
current loss is:  tensor(-18.3308, device='cuda:1')
current relative error is:  tensor(0.0623, device='cuda:1')
current epoch is:  18300
current loss is:  tensor(-19.8470, device='cuda:1')
current relative error is:  tensor(0.0569, device='cuda:1')
current epoch is:  18350
current loss is:  tensor(-19.2508, device='cuda:1')
current relative error is:  tensor(0.0582, device='cuda:1')
current epoch is:  18400
current loss is:  tensor(-19.9419, device='cuda:1')
current relative error is:  tensor(0.0588, device='cuda:1')
current epoch is:  18450
current loss is:  tensor(-18.9063, device='cuda:1')
current relative error is:  tensor(0.0578, device='cuda:1')
current epoch is:  18500
current loss is:

In [19]:
error_save_append = np.zeros(traintime)
time_start = time.time()
for i in range(traintime):
    optimizer.zero_grad()
    x = Gendata()
    x.requires_grad = True
    losses = DRM(x)
    losses.backward()
    optimizer.step()
    error = relative_error()
    error_save_append[i] = float(error)
    
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", losses.detach())
        print("current relative error is: ", error.detach())
        np.save("DRM_relative_error_4D_Robin_batch_size_4000_append.npy", error_save_append)
np.save("DRM_relative_error_4D_Robin_batch_size_4000_append.npy", error_save_append)
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(-19.4988, device='cuda:1')
current relative error is:  tensor(0.0554, device='cuda:1')
current epoch is:  50
current loss is:  tensor(-19.2909, device='cuda:1')
current relative error is:  tensor(0.0544, device='cuda:1')
current epoch is:  100
current loss is:  tensor(-19.2149, device='cuda:1')
current relative error is:  tensor(0.0553, device='cuda:1')
current epoch is:  150
current loss is:  tensor(-18.6595, device='cuda:1')
current relative error is:  tensor(0.0547, device='cuda:1')
current epoch is:  200
current loss is:  tensor(-18.9416, device='cuda:1')
current relative error is:  tensor(0.0547, device='cuda:1')
current epoch is:  250
current loss is:  tensor(-18.9982, device='cuda:1')
current relative error is:  tensor(0.0571, device='cuda:1')
current epoch is:  300
current loss is:  tensor(-19.3805, device='cuda:1')
current relative error is:  tensor(0.0525, device='cuda:1')
current epoch is:  350
current loss is:  tensor(-18.8655, 

current epoch is:  3050
current loss is:  tensor(-19.4281, device='cuda:1')
current relative error is:  tensor(0.0505, device='cuda:1')
current epoch is:  3100
current loss is:  tensor(-19.2957, device='cuda:1')
current relative error is:  tensor(0.0472, device='cuda:1')
current epoch is:  3150
current loss is:  tensor(-18.5017, device='cuda:1')
current relative error is:  tensor(0.0468, device='cuda:1')
current epoch is:  3200
current loss is:  tensor(-19.2324, device='cuda:1')
current relative error is:  tensor(0.0487, device='cuda:1')
current epoch is:  3250
current loss is:  tensor(-19.3058, device='cuda:1')
current relative error is:  tensor(0.0460, device='cuda:1')
current epoch is:  3300
current loss is:  tensor(-19.2744, device='cuda:1')
current relative error is:  tensor(0.0479, device='cuda:1')
current epoch is:  3350
current loss is:  tensor(-20.6748, device='cuda:1')
current relative error is:  tensor(0.0470, device='cuda:1')
current epoch is:  3400
current loss is:  tensor

current epoch is:  6100
current loss is:  tensor(-19.6029, device='cuda:1')
current relative error is:  tensor(0.0438, device='cuda:1')
current epoch is:  6150
current loss is:  tensor(-19.2063, device='cuda:1')
current relative error is:  tensor(0.0436, device='cuda:1')
current epoch is:  6200
current loss is:  tensor(-18.3223, device='cuda:1')
current relative error is:  tensor(0.0436, device='cuda:1')
current epoch is:  6250
current loss is:  tensor(-18.8172, device='cuda:1')
current relative error is:  tensor(0.0454, device='cuda:1')
current epoch is:  6300
current loss is:  tensor(-19.4004, device='cuda:1')
current relative error is:  tensor(0.0425, device='cuda:1')
current epoch is:  6350
current loss is:  tensor(-18.9938, device='cuda:1')
current relative error is:  tensor(0.0443, device='cuda:1')
current epoch is:  6400
current loss is:  tensor(-21.0525, device='cuda:1')
current relative error is:  tensor(0.0426, device='cuda:1')
current epoch is:  6450
current loss is:  tensor

current epoch is:  9150
current loss is:  tensor(-18.7441, device='cuda:1')
current relative error is:  tensor(0.0397, device='cuda:1')
current epoch is:  9200
current loss is:  tensor(-20.0157, device='cuda:1')
current relative error is:  tensor(0.0419, device='cuda:1')
current epoch is:  9250
current loss is:  tensor(-19.1626, device='cuda:1')
current relative error is:  tensor(0.0398, device='cuda:1')
current epoch is:  9300
current loss is:  tensor(-18.2872, device='cuda:1')
current relative error is:  tensor(0.0463, device='cuda:1')
current epoch is:  9350
current loss is:  tensor(-19.2351, device='cuda:1')
current relative error is:  tensor(0.0394, device='cuda:1')
current epoch is:  9400
current loss is:  tensor(-19.0124, device='cuda:1')
current relative error is:  tensor(0.0407, device='cuda:1')
current epoch is:  9450
current loss is:  tensor(-19.7760, device='cuda:1')
current relative error is:  tensor(0.0390, device='cuda:1')
current epoch is:  9500
current loss is:  tensor

current epoch is:  12150
current loss is:  tensor(-18.3774, device='cuda:1')
current relative error is:  tensor(0.0364, device='cuda:1')
current epoch is:  12200
current loss is:  tensor(-19.7933, device='cuda:1')
current relative error is:  tensor(0.0363, device='cuda:1')
current epoch is:  12250
current loss is:  tensor(-19.5924, device='cuda:1')
current relative error is:  tensor(0.0377, device='cuda:1')
current epoch is:  12300
current loss is:  tensor(-19.3304, device='cuda:1')
current relative error is:  tensor(0.0387, device='cuda:1')
current epoch is:  12350
current loss is:  tensor(-18.7815, device='cuda:1')
current relative error is:  tensor(0.0354, device='cuda:1')
current epoch is:  12400
current loss is:  tensor(-18.5721, device='cuda:1')
current relative error is:  tensor(0.0354, device='cuda:1')
current epoch is:  12450
current loss is:  tensor(-19.4514, device='cuda:1')
current relative error is:  tensor(0.0383, device='cuda:1')
current epoch is:  12500
current loss is:

current epoch is:  15150
current loss is:  tensor(-19.9682, device='cuda:1')
current relative error is:  tensor(0.0358, device='cuda:1')
current epoch is:  15200
current loss is:  tensor(-19.3730, device='cuda:1')
current relative error is:  tensor(0.0337, device='cuda:1')
current epoch is:  15250
current loss is:  tensor(-20.2762, device='cuda:1')
current relative error is:  tensor(0.0326, device='cuda:1')
current epoch is:  15300
current loss is:  tensor(-18.4894, device='cuda:1')
current relative error is:  tensor(0.0377, device='cuda:1')
current epoch is:  15350
current loss is:  tensor(-19.0349, device='cuda:1')
current relative error is:  tensor(0.0337, device='cuda:1')
current epoch is:  15400
current loss is:  tensor(-19.3343, device='cuda:1')
current relative error is:  tensor(0.0328, device='cuda:1')
current epoch is:  15450
current loss is:  tensor(-19.6641, device='cuda:1')
current relative error is:  tensor(0.0336, device='cuda:1')
current epoch is:  15500
current loss is:

current epoch is:  18150
current loss is:  tensor(-18.8937, device='cuda:1')
current relative error is:  tensor(0.0313, device='cuda:1')
current epoch is:  18200
current loss is:  tensor(-19.3938, device='cuda:1')
current relative error is:  tensor(0.0311, device='cuda:1')
current epoch is:  18250
current loss is:  tensor(-18.9999, device='cuda:1')
current relative error is:  tensor(0.0322, device='cuda:1')
current epoch is:  18300
current loss is:  tensor(-19.6388, device='cuda:1')
current relative error is:  tensor(0.0320, device='cuda:1')
current epoch is:  18350
current loss is:  tensor(-20.0074, device='cuda:1')
current relative error is:  tensor(0.0312, device='cuda:1')
current epoch is:  18400
current loss is:  tensor(-19.9611, device='cuda:1')
current relative error is:  tensor(0.0304, device='cuda:1')
current epoch is:  18450
current loss is:  tensor(-18.9240, device='cuda:1')
current relative error is:  tensor(0.0335, device='cuda:1')
current epoch is:  18500
current loss is: